# 过拟合

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D

# Default parameters for plots
matplotlib.rcParams['font.size'] = 20
matplotlib.rcParams['figure.titlesize'] = 20
matplotlib.rcParams['figure.figsize'] = [12, 10]
matplotlib.rcParams['font.family'] = ['SimHei']# ['Noto Sans CJK JP']
matplotlib.rcParams['axes.unicode_minus']=False 
%matplotlib inline

gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
except RuntimeError as e:
    print(e)
    

## 1 模型容量
通俗地讲，模型的容量或表达能力，是指模型**拟合复杂函数**的能力。一种体现模型容量的指标为模型的假设空间(Hypothesis Space)大小，即模型可以表示的函数集的大小。假设空间越大越完备，从假设空间中搜索出逼近真实模型的函数也就越有可能；反之，如果假设空间非常受限，就很难从中找到逼近真实模型的函数。

函数的假设空间越大，就越有可能找到一个函数更好地逼近真实分布的函数模型. 但是过大的假设空间无疑会增加搜索难度和计算代价. 由于观测误差的存在，较大的假设空间中可能包含了大量表达能力过强的函数，能够将训练样本的观测误差也学习进来，从而伤害了模型的泛化能力.


## 2 欠拟合和过拟合

当模型在训练集上误差一直维持较高的状态，很难优化减少，同时在测试集上也表现不佳时，我们可以考虑是否出现了欠拟合的现象. 这个时候可以通过增加神经网络的层数、增大中间维度的大小等手段，比较好的解决欠拟合的问题.

当模型在训练集上具有较低的误差，在测试集上具有较高的误差时, 此时模型可能出于过拟合状态. 现代深度神经网络中过拟合现象非常容易出现，主要是因为神经网络的表达能力非常强，训练集样本数不够，很容易就出现了神经网络的容量偏大的现象.

## 3 抑制过拟合

### 3.1 数据集划分:   
将数据集划分为: 
   * 训练集: 用于训练模型的参数
   * 验证集: 用于选择模型的超参数(模型选择，Model selection):
       + 根据验证集的性能表现来调整学习率、权值衰减系数、训练次数等。
       + 根据验证集的性能表现来重新调整网络拓扑结构。 
       + 根据验证集的性能表现判断是否过拟合和欠拟合
   
   * 测试集: 用来评价模型

### 3.2 提前停止(early stopping)

一般把对训练集中的一个Batch 运算更新一次叫做一个Step，对训练集的所有样本循环迭代一次叫做一个Epoch。验证集可以在数次Step 或数次Epoch 后使用，计算模型的验证性能。验证的步骤过于频繁，能够精准地观测模型的训练状况，但是也会引入额外的计算代价，一般建议几个Epoch 后进行一次验证运算。

当训练有足够的表示能力甚至会过拟合的大模型时, 我们经常观察到, 训练误差会随着时间的推移逐渐降低但验证集的误差会再次上升.

这意味着，对于神经网络，即使网络结构超参数保持不变(即网络最大容量固定)，模型依然可能会出现过拟合的现象，这是因为神经网络的有效容量和网络参数的状态息息相关，神经网络的有效容量可以很大，也可以通过稀疏化参数、添加正则化等手段降低有效容量。在训练的前中期，神经网络的过拟合现象没有出现，当随着训练Epoch数的增加，过拟合程度越来越严重.

**提前停止**: 只要返回使验证集误差最低的参数设置, 就可以获得验证集误差更低的模型(并且因此有希望获得更好的测试误差). 具体地，对于分类问题，我们可以记录模型的验证准确率，并监控验证准确率的变化，当发现验证准确率连续𝑛个Epoch 没有下降时，可以预测可能已经达到了最适合的Epoch附近，从而提前终止训练。

## 4 模型设计

对于神经网络来说，网络的层数和参数量是网络容量很重要的参考指标，通过减少网络的层数，并减少每层中网络参数量的规模，可以有效降低网络的容量。反之，如果发现模型欠拟合，需要增大网络的容量，可以通过增加层数，增大每层的参数量等方式实现。

## 5 正则化
通过设计不同层数、大小的网络模型可以为优化算法提供初始的函数假设空间，但是模型的实际容量可以随着网络参数的优化更新而产生变化.例如多项式函数模型:
$$
y = \beta_0 + \beta_1 x + \beta_2 x^2 + ... + \beta_n x^n + \epsilon
$$
模型的容量可以通过𝑛简单衡量。在训练的过程中，如果网络参数$\beta_{k+1}, ⋯ , \beta_𝑛$均为0，
那么网络的实际容量退化到𝑘次多项式的函数容量。因此，通过限制网络参数的**稀疏性**，可以来约束网络的实际容量.

对模型的参数添加额外的约束后，优化的目标变为:
$$
min L(f_{\theta}(x), y) + \lambda \cdot \Omega(\theta), \quad (x, y)\in D^{train}
$$
一般地, 参数$\theta$的稀疏性约束通过约束参数$\theta$的L范数实现
$$
\Omega(\theta) = \sum_{\theta_i}||\theta_i||_l
$$
优化算法会在降低L(x, y)的同时，尽可能地迫使网络参数$\theta_i$变得稀疏

- L0正则化: L0范数$||\theta_i‖$0定义为$\theta_i$中非零元素的个数, 由于L0范数并不可导，不能利用梯度下降算法进行优化，在神经网络中使用的并不多.

- L1正则化: L1范数$||\theta_i‖$0定义为张量$\theta_i$中所有元素的绝对值之和. L1正则化也叫Lasso Regularization, 它是连续可导的.

- L2正则化: L2范数$||\theta_i‖$0定义为张量$\theta_i$中所有元素的平方和. L1正则化也叫Ridge Regularization, 也是连续可导的.

In [ ]:
from tensorflow.keras import layers, losses, Sequential, regularizers


In [ ]:
x = tf.random.normal([3, 2])
x

In [ ]:
# L1 正则化项

w1 = tf.random.normal([4, 3])
w2 = tf.random.normal([4, 2])

loss_reg = tf.reduce_sum(tf.math.abs(w1)) + tf.reduce_sum(tf.math.abs(w2))

In [ ]:
# L2 正则化项

w1 = tf.random.normal([4, 3])
w2 = tf.random.normal([4, 2])

loss_reg = tf.reduce_sum(tf.square(w1)) + tf.reduce_sum(tf.square(w2))

In [ ]:
# 手动计算 添加正则化项的损失
lambda_ = 0.01
loss = lambda_ * tf.reduce_sum(tf.square(w1))

In [ ]:
# 在层方式时添加范数函数
fc1 = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(lambda_))

## 6 Bagging
**Bagging**是并行集成学习方法最著名的代表, 直接基于自助采样法(bootstrap sampling, 亦称可重复采样法或有放回采样法).主要想法是分别训练几个不同的模型, 然后让所有的模型表决测试样例的输出. 这是机器学习中常规策略的一个例子, 被称为**模型平均**(model averaging). 采用这种策略的技术就被称为集成方法.

模型平均奏效的原因是不同的模型通常不会在测试集上产生完全相同的误差.

## 7 Dropout
Dropout 通过随机断开神经网络的连接，减少每次训练时实际参与计算的模型的参数量；但是在测试时，Dropout 会恢复所有的连接，保证模型测试时获得最好的性能。
具体而言, Dropout训练的集成包括所有从基础网络除去非输出单元后形成的子网络, 只需要将一些单元的输出乘零就能有效地删除一个单元. 
Dropout训练与Bagging训练的区别: 在Bagging的情况下, 所有模型都是独立的. 在Dropout的情况下, 所有模型共享参数, 其中每个模型继承父神经网络参数的不同子集.

dropout 的理解：

不要依赖于任何一个特征，因为该单元的输入可能随时被清除，因此该单元通过这种方式传播下去，并为单元的四个输入增加一点权重，通过传播所有权重，dropout将产生收缩权重的平方范数的效果，和之前讲的L2正则化类似；实施dropout的结果实它会压缩权重，并完成一些预防过拟合的外层正则化；L2对不同权重的衰减是不同的，它取决于激活函数倍增的大小。  

缺点：

dropout一大缺点就是代价函数J不再被明确定义，每次迭代，都会随机移除一些节点，如果再三检查梯度下降的性能，实际上是很难进行复查的。

In [ ]:
tf.nn.dropout?

In [ ]:
# 添加dropout 操作，断开概率为0.5
x = tf.nn.dropout(x, rate=0.5)
x  # scaled up by `1 / (1 - rate)

In [ ]:
# 添加Dropout 层，断开概率为0.5
network = Sequential()
network.add(layers.Dropout(rate=0.5))
network.build(input_shape=(None, 2))
network.summary()

## 8 数据增强
增加数据集规模是解决过拟合最重要的途径.数据增强(DataAugmentation)是指在维持样本标签不变的条件下，根据先验知识改变样本的特征，使得新产生的样本也符合或者近似符合数据的真实分布。

以图片数据为例, 旋转、缩放、平移、裁剪、改变视角、遮挡某局部区域都不会改变图片的主体类别标签，因此针对图片数据，可以有多种数据增强方式。

In [ ]:
raw_image = tf.io.read_file('lenna.png')  # 读取文件  返回tf.string格式
raw_image.dtype, raw_image.shape

In [ ]:
x_ori = tf.image.decode_image(raw_image, channels=3, dtype=tf.uint8)  # 
x = tf.image.resize(x_ori, [244, 244], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
x.shape, x.dtype

In [ ]:
x[1, 1, :]

旋转

In [ ]:
x1 = tf.image.rot90(x, k=1)  # 逆时针旋转k个90度
x.shape

In [ ]:
image = tf.io.encode_jpeg(x1)
tf.io.write_file('lenna_rot1.png', image)
del x1

翻转

In [ ]:
# 随机翻转
# x_2 = tf.image.random_flip_left_right(x)
# 水平翻转
x1 = tf.image.flip_left_right(x)
image = tf.io.encode_jpeg(x1)
tf.io.write_file('lenna_flip1.png', image)

In [ ]:
# 竖直翻转
x_1 = tf.image.flip_up_down(x)
# x_3 = tf.image.random_flip_up_down(x)
image = tf.io.encode_jpeg(x_1)
tf.io.write_file('lenna_flip2.png', image)

裁剪

In [ ]:
# 图片先缩放到稍大尺寸
x_1 = tf.image.resize(x, [270, 270], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
# 随机裁剪
x_1 = tf.image.random_crop(x_1, [244, 244, 3])
image = tf.io.encode_jpeg(x_1)
tf.io.write_file('lenna_crop1.png', image)

In [ ]:
# 中心裁剪
x_1 = tf.image.central_crop(x, central_fraction=0.8)
image = tf.io.encode_jpeg(x_1)
tf.io.write_file('lenna_crop_central.png', image)

In [ ]:
tf.random.normal([244, 244, 3]) * 0.2

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(x.numpy())

In [ ]:
# 添加高斯噪声 sigma^2=10
x_1 = (x.numpy() + np.random.randn(244, 244, 3) * 10) / 255
plt.figure(figsize=(12, 10))                                                                                                                                                                                                                                                                                                                                             
plt.imshow(x_1)

## 过拟合问题

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

N_SAMPLES = 1000
X, y = make_moons(n_samples=N_SAMPLES, noise=0.25, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
def make_plot(X, y, title, file_name=None, xx=None, yy=None, preds=None):
    plt.figure(figsize=(10, 8))
    axes = plt.gca()
    xmax, xmin = np.max(X[:, 0]), np.min(X[:, 0])
    ymax, ymin = np.max(X[:, 1]), np.min(X[:, 1])
    axes.set_xlim([xmin, xmax])
    axes.set_ylim([ymin, ymax])
    if (xx is not None and yy is not None and preds is not None):
        plt.contour(xx, yy, preds.reshape(xx.shape), 25, alpha=0.08, cmap=plt.cm.Spectral)
        plt.contourf(xx, yy, preds.reshape(xx.shape), levels=[.5, 1.], cmap='Greys', vmin=0, vmax=.6)
    
    plt.scatter(X[y==0, 0], X[y==0, 1], s=10, cmap=plt.cm.Spectral, marker='s')
    plt.scatter(X[y==1, 0], X[y==1, 1], s=10,cmap=plt.cm.Spectral, marker='o')
    plt.title(title)
    if file_name is not None:
        plt.savefig(file_name)

In [ ]:
make_plot(X, y, title='原始')

**网络层的影响**

探讨不同的网络深度下的过拟合程度

In [ ]:
N_EPOCHS = 500

In [ ]:
xx, yy = np.mgrid[-2:3:0.05, -2:2:0.05]
xx.shape, yy.shape

In [ ]:
np.c_[xx.ravel(), yy.ravel()]

In [ ]:
for n in range(5):
    model = Sequential()
    model.add(layers.Dense(8, input_dim=2, activation='relu'))
    for _ in range(n):  # 共 n + 2
        model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    # 模型装配
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # 训练
    history = model.fit(X_train, y_train, epochs=N_EPOCHS, verbose=0)
    xx, yy = np.mgrid[-2:3:0.02, -2:2:0.02]
    preds = model.predict_classes(np.c_[xx.ravel(), yy.ravel()])
    name = f"隐藏网络层数{n}"
    file_name = f"隐藏网络层数{n}.png"
    make_plot(X_train, y_train, name, file_name, xx, yy, preds)

**Dropout的影响**

7层全连接层, 5次试验, 分别添加0~4层的Dropout层, 每次500个epoch


In [ ]:

for n in range(5):
    model = Sequential()
    model.add(layers.Dense(8, input_dim=2, activation='relu'))  # 输入层
    drop_count = 0
    for _ in range(5):  # 总共7层 ５+ 1 + 1
        model.add(layers.Dense(64, activation='relu'))
        if drop_count < n:  # 分别添加0-4层Dropout层
            drop_count += 1
            model.add(layers.Dropout(rate=0.5))
    model.add(layers.Dense(1, activation='sigmoid'))  # 输出层
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=N_EPOCHS, verbose=0)
    xx, yy = np.mgrid[-2:3:0.02, -2:2:0.02]
    preds = model.predict_classes(np.c_[xx.ravel(), yy.ravel()])
    name = f"Dropout层数{n}"
    file_name = f"Dropout层数{n}.png"
    make_plot(X_train, y_train, name, file_name, xx, yy, preds)

In [ ]:
w1, b1, w2, b2, w3, b3, *_ = model.weights
w3.shape, w2.shape

In [ ]:
xx, yy = np.mgrid[0:32:1, 0:32:1]
xx

In [ ]:
w3

In [ ]:
plt.cm.Spectral?

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
zz = np.sin(xx)
ax.plot_surface(xx, yy, zz,  rstride=1, cstride=1, cmap=plt.get_cmap('rainbow'))  #

**正则化系数的影响**

使用L2正则项, 构建了5 层的神经网络，其中第2、3、4 层神经网络层的权值张量W 均添加L2 正则化约束项

In [ ]:
def plot_weights(model, title, file_name):
    # 获取第三层系数 w 64*64
    _, _, _, _, w3, *_ = model.weights
    w3 = w3.numpy()
    print(f'{title}: max:{np.max(w3)}, min:{np.min(w3)}, mean:{np.mean(w3)}, var:{w3.var()}')
    xx, yy = np.mgrid[0:64:1, 0:64:1]
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_surface(xx, yy, w3, rstride=1, cstride=1, cmap=plt.cm.rainbow)
    ax.set_title(title)
    plt.savefig(file_name)

In [ ]:
_lambdas = [1e-5, 1e-3, 1e-1, 0.12, 0.2] 
def build_model_with_regularization(_lambda):
    model = Sequential()
    model.add(layers.Dense(8, activation='relu'))
    for _ in range(3):
        model.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(_lambda)))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
for i, _lambda in enumerate(_lambdas):
    model = build_model_with_regularization(_lambda)
    model.fit(X_train, y_train, epochs=N_EPOCHS, verbose=0)
    name = f"正则化系数{_lambda}"
    file_name = f"w系数{i}"
    plot_weights(model, name, file_name)
    xx, yy = np.mgrid[-2:3:0.02, -2:2:0.02]
    preds = model.predict_classes(np.c_[xx.ravel(), yy.ravel()])
    file_name = f"决策平面{i}"
    make_plot(X_train, y_train, name, file_name, xx, yy, preds)